<a href="https://colab.research.google.com/github/roytalari/SparkPySpark/blob/main/Lab2SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark - Spark SQL & Dataframes

## Install Spark
## Set Environment variables
## Create a Spark Session


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.6.tgz
!tar -xvf spark-2.4.3-bin-hadoop2.6.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.6"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Test Spark

In [4]:
df = spark.createDataFrame([{"Google": "Colab","Spark":"Scala"},{"Google": "Runbook","Spark":"PySpark"}])
df.show()

/content/spark-2.4.3-bin-hadoop2.6/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-------+-------+
| Google|  Spark|
+-------+-------+
|  Colab|  Scala|
|Runbook|PySpark|
+-------+-------+



## COPY A DATA FILE TO YOUR LOCAL COLAB ENVIRONMENT

In [5]:
!wget https://raw.githubusercontent.com/futurexskill/bigdata/master/retailstore.csv

--2020-10-24 20:24:46--  https://raw.githubusercontent.com/futurexskill/bigdata/master/retailstore.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279 [text/plain]
Saving to: ‘retailstore.csv’

retailstore.csv     100%[===================>]     279  --.-KB/s    in 0s      

2020-10-24 20:24:46 (5.41 MB/s) - ‘retailstore.csv’ saved [279/279]



# Check if the file is copied

In [6]:
!ls

retailstore.csv  spark-2.4.3-bin-hadoop2.6
sample_data	 spark-2.4.3-bin-hadoop2.6.tgz


# **  DataFrame **

# Read the CSV file into a DataFrame

In [7]:
customerDF = spark.read.csv("retailstore.csv", header = "True")

In [7]:
type(customerDF)

pyspark.sql.dataframe.DataFrame

# Print the dataset

In [8]:
customerDF.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



In [9]:
customerDF.show(5)

+---+------+------+-------+---------+
|Age|Salary|Gender|Country|Purchased|
+---+------+------+-------+---------+
| 18| 20000|  Male|Germany|        N|
| 19| 22000|Female| France|        N|
| 20| 24000|Female|England|        N|
| 21|  null|  Male|England|        N|
| 22| 50000|  Male| France|        Y|
+---+------+------+-------+---------+
only showing top 5 rows



# Statiscal Information about the dataframe  gives mean, median of mainly numeric columns

In [10]:
customerDF.describe().show()

+-------+-----------------+-----------------+------+-------+---------+
|summary|              Age|           Salary|Gender|Country|Purchased|
+-------+-----------------+-----------------+------+-------+---------+
|  count|                9|                8|    10|     10|       10|
|   mean|22.11111111111111|          31875.0|  null|   null|     null|
| stddev|2.934469476943168|9818.895777311942|  null|   null|     null|
|    min|               18|            20000|Female|England|        N|
|    max|               27|            50000|  Male|Germany|        Y|
+-------+-----------------+-----------------+------+-------+---------+



In [11]:
customerDF.groupBy("country").count().show()

+-------+-----+
|country|count|
+-------+-----+
|Germany|    3|
| France|    4|
|England|    3|
+-------+-----+



In [12]:
customerDF.groupby("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|Female|    5|
|  Male|    5|
+------+-----+



# Running SQL Queries programmatically

### Create a temporary table

In [13]:
customerDF.createOrReplaceTempView("customer")

# Fetch all records from the table using a SQL query 

In [14]:
results = spark.sql("select * from customer")

In [15]:
type(results)

pyspark.sql.dataframe.DataFrame

In [16]:
results.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



# **Spark Dataframe** 

## The sql function on a SparkSession enables applications to run SQL queries prgramtically and return the results as a Dataframe.

In [17]:
new_results = spark.sql("select * from customer where age>22").show()

+---+------+------+-------+---------+
|Age|Salary|Gender|Country|Purchased|
+---+------+------+-------+---------+
| 23| 35000|Female|England|        N|
| 24|  null|  Male|Germany|        N|
| 25| 32000|Female| France|        Y|
| 27| 37000|Female| France|        N|
+---+------+------+-------+---------+



In [18]:
new_results = spark.sql("select * from customer where age>22").select("salary").show()

+------+
|salary|
+------+
| 35000|
|  null|
| 32000|
| 37000|
+------+



### We can apply filter method to filter data

In [19]:
customerDF.filter("salary > 20000").show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



### Group By gender find average salary and max age

In [21]:
customerDF.groupBy("gender").agg({"salary":"average", "age":"max"}).show()

+------+-----------+--------+
|gender|avg(salary)|max(age)|
+------+-----------+--------+
|Female|    30000.0|      27|
|  Male|    35000.0|      24|
+------+-----------+--------+



### Adding new column

In [22]:
customerDF.withColumn('doublesalary', customerDF['salary']*2).show()

+----+------+------+-------+---------+------------+
| Age|Salary|Gender|Country|Purchased|doublesalary|
+----+------+------+-------+---------+------------+
|  18| 20000|  Male|Germany|        N|     40000.0|
|  19| 22000|Female| France|        N|     44000.0|
|  20| 24000|Female|England|        N|     48000.0|
|  21|  null|  Male|England|        N|        null|
|  22| 50000|  Male| France|        Y|    100000.0|
|  23| 35000|Female|England|        N|     70000.0|
|  24|  null|  Male|Germany|        N|        null|
|  25| 32000|Female| France|        Y|     64000.0|
|null| 35000|  Male|Germany|        N|     70000.0|
|  27| 37000|Female| France|        N|     74000.0|
+----+------+------+-------+---------+------------+



### Renaming a column 

In [23]:
customerDF.withColumnRenamed('salary','income').show()

+----+------+------+-------+---------+
| Age|income|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



In [24]:
customerDF.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



#### Drop null values with na

data cleasning operations

In [25]:
customerDF.na.drop().show()

+---+------+------+-------+---------+
|Age|Salary|Gender|Country|Purchased|
+---+------+------+-------+---------+
| 18| 20000|  Male|Germany|        N|
| 19| 22000|Female| France|        N|
| 20| 24000|Female|England|        N|
| 22| 50000|  Male| France|        Y|
| 23| 35000|Female|England|        N|
| 25| 32000|Female| France|        Y|
| 27| 37000|Female| France|        N|
+---+------+------+-------+---------+



In [26]:
customerDF.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



# Explore other methods in pyspark documentation! 
*Thank you*